## Notebook for making predictions with deleted constant y samples


In [392]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

### Reading dataset stored in other notebooks

In [393]:
sys.path.append('../')

%store -r dml

dm = dml

## Getting data

In [394]:
print(dm.data_A.columns)

X_A_train = dm.data_A.iloc[:, 2:]
Y_A_train = dm.data_A.iloc[:, 1]

X_B_train = dm.data_B.iloc[:, 2:]
Y_B_train = dm.data_B.iloc[:, 1]

X_C_train = dm.data_C.iloc[:, 2:]
Y_C_train = dm.data_C.iloc[:, 1]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_C_test = dm.X_test_estimated_c[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]

#X_A_train['direct_rad:W_lag_12']

Index(['date_forecast', 'pv_measurement', 'absolute_humidity_2m:gm3',
       'air_density_2m:kgm3', 'clear_sky_energy_1h:J', 'clear_sky_rad:W',
       'dew_or_rime:idx', 'dew_point_2m:K', 'diffuse_rad:W',
       'diffuse_rad_1h:J',
       ...
       'direct_rad:W_lag_-504', 'direct_rad:W_lag_-528',
       'direct_rad:W_lag_-552', 'direct_rad:W_lag_-576',
       'direct_rad:W_lag_-600', 'direct_rad:W_lag_-624',
       'direct_rad:W_lag_-648', 'direct_rad:W_lag_-672',
       'direct_rad:W_lag_-696', 'direct_rad:W_lag_-720'],
      dtype='object', length=102)


## Catboost models
will try with two models:
-   One for dataset A
-   One for B and C

In [395]:
catModel_a = CatBoostRegressor()
catModel_b = CatBoostRegressor()
catModel_c = CatBoostRegressor()

#catModel_bc = CatBoostRegressor()

In [396]:
X_BC_train = pd.concat([X_B_train, X_C_train])
Y_BC_train = pd.concat([Y_B_train, Y_C_train])

In [397]:
catModel_a.fit(X_A_train,Y_A_train)
catModel_b.fit(X_B_train,Y_B_train)
catModel_c.fit(X_C_train,Y_C_train)

#catModel_bc.fit(X_BC_train,Y_BC_train)


Learning rate set to 0.067297
0:	learn: 1246.5719784	total: 26.6ms	remaining: 26.5s
1:	learn: 1185.4537680	total: 34.2ms	remaining: 17.1s
2:	learn: 1128.2535691	total: 41.6ms	remaining: 13.8s
3:	learn: 1075.1025970	total: 48.9ms	remaining: 12.2s
4:	learn: 1027.5690119	total: 55.9ms	remaining: 11.1s
5:	learn: 982.6147345	total: 62.4ms	remaining: 10.3s
6:	learn: 941.7633431	total: 69.1ms	remaining: 9.8s
7:	learn: 904.3748248	total: 75.6ms	remaining: 9.37s
8:	learn: 867.8214100	total: 82.1ms	remaining: 9.04s
9:	learn: 834.8184663	total: 89.2ms	remaining: 8.83s
10:	learn: 805.6256391	total: 95.7ms	remaining: 8.61s
11:	learn: 778.7362665	total: 102ms	remaining: 8.43s
12:	learn: 753.3106896	total: 109ms	remaining: 8.28s
13:	learn: 729.8503369	total: 116ms	remaining: 8.17s
14:	learn: 709.1998551	total: 122ms	remaining: 8.03s
15:	learn: 690.5032875	total: 129ms	remaining: 7.93s
16:	learn: 674.2570233	total: 135ms	remaining: 7.83s
17:	learn: 659.1305590	total: 142ms	remaining: 7.75s
18:	learn: 

In [398]:
# Feature importance
catModel_a.get_feature_importance(prettified=True)

,Feature Id,Importances
0,direct_rad:W,11.845859
1,direct_rad:W_lag_-1,9.767393
2,sum_rad:W,5.586389
3,clear_sky_rad:W,4.763634
4,direct_rad:W_lag_-2,4.018548
...,...,...
95,elevation:m,0.000000
96,fresh_snow_3h:cm,0.000000
97,prob_rime:p,0.000000
98,snow_drift:idx,0.000000


## Preds

In [399]:
pred_A = catModel_a.predict(X_A_test)
pred_B = catModel_b.predict(X_B_test)
pred_C = catModel_c.predict(X_C_test)

#pred_B = catModel_bc.predict(X_B_test)
#pred_C = catModel_bc.predict(X_C_test)

In [400]:
df_A = pd.DataFrame()

df_A["predict"] = pred_A
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["predict"] = pred_B
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["predict"] = pred_C
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df = df.drop("location", axis=1)

df[df<0] = 0

# NAME THE FILE 
df.to_csv("./ext_preds/catboost_lag_18.csv")

df.shape

(2160, 1)